# Perceptions&Interactions TP3 - Traitement d'image, Partie 2

## Introduction

*Imports et chargement d'image*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

img = cv2.imread("butterfly_gray.png")
plt.imshow(img)
plt.show()

### **0. Filtre médian**

Codez la fonction filtre_median(img, half_size).

Cette fonction renvoie une nouvelle image dans laquelle chaque pixel (x,y) équivaut à la valeur médiane parmi les pixels voisins. Le voisinage considéré pourra être un carré allant de (x-half_size, y-half_size) à (x+half_size, y+half_size), c'est-à-dire un carré centré en (x-y) de taille size=(2\*half_size)+1.

On considèrera que les 3 canaux sont identiques (image en niveau de gris sur 3 canaux).

### **1. Erosion**

Codez la fonction erosion(img, half_size).

Cette fonction renvoie une nouvelle image dans laquelle chaque pixel (x,y) équivaut à la valeur la plus **faible** parmi les pixels voisins. Le voisinage considéré pourra être un carré allant de (x-half_size, y-half_size) à (x+half_size, y+half_size), c'est-à-dire un carré centré en (x-y) de taille size=(2\*half_size)+1.

On considèrera que les 3 canaux sont identiques (image en niveau de gris sur 3 canaux).

In [ ]:
def erosion(img, half_size):
    result = img.copy()
    #...
    return result


# kernel = np.array([[1, 1, 1],
#                    [1, 1, 1],
#                    [1, 1, 1]], dtype=np.uint8)

# eroded_image = cv2.erode(img, kernel, iterations=1)

img_erodee = erosion(img, 2)
plt.imshow(img_erodee)

### **2. Dilatation**

Codez la fonction dilation(img, half_size).

Cette fonction, à l'instar de la fonction erosion, renvoie une nouvelle image dans laquelle chaque pixel (x,y) équivaut à la valeur la plus **élevée** parmi les pixels voisins.

On considèrera à nouveau que les 3 canaux sont identiques (image en niveau de gris sur 3 canaux).

In [ ]:
def dilation(img, half_size):
    result = img.copy()
    #...
    return result


img_dilatee = erosion(img, 2)
plt.imshow(img_dilatee)

### 3. Extraction de contours avec dilation et erosion

In [ ]:
g1 = img - img_erodee #gradient interne, pixels retirés par l'érosion = contours intérieurs
g2 = img_dilatee - img #gradient externe, pixels ajoutés par la dilatation = contours extérieurs
g3 = img_dilatee - img_erodee #gradient morphologique, combinaison des deux approches = tous les contours


fig, ax = plt.subplots(2,2, figsize=(15,10))
ax[0,0].imshow(img)
ax[0,1].imshow(g1)
ax[1,0].imshow(g2)
ax[1,1].imshow(g3)
[ax[x,y].axis("off") for x in range(2) for y in range(2)];


### 4. Features from Accelerated Segment Test (FAST)

FAST est un algorithme de détection de points d'intérêt. Pour chaque pixel, un cercle de rayon 3 des pixels "voisins" est calculé. Tout d'abord, on choisit de définir un cercle tel que :

In [ ]:
def bresenham_circle():
    circle = np.zeros((7,7))
    for i in range(circle.shape[0]):
        for j in range(circle.shape[1]):
            distance = np.sqrt((3-i)*(3-i) + (3-j)*(3-j))
            if distance > 2.5 and distance < 3.5:
                circle[i, j] = 1
    return circle
plt.imshow(bresenham_circle())

Ce cercle pourra ensuite être utilisé de manière similaire aux noyaux de la fonction convolution. Le cercle est composé de 16 pixels. Pour chaque pixel du cercle, on vérifie si sa différence d'intensité avec le pixel du centre du cercle est supérieure au threshold donné en paramètre. Si au moins 12 des 16 pixels du cercle ont une différence supérieure à ce threshold, alors le pixel est considéré comme un point d'intérêt.

In [ ]:
crew_d = cv2.imread("Crew_Dragon.jpg", cv2.IMREAD_GRAYSCALE)
capsule = cv2.imread("capsule.jpg", cv2.IMREAD_GRAYSCALE)

fast = cv2.FastFeatureDetector_create(threshold=20)

kp_cd = fast.detect(crew_d,None)
img2 = cv2.drawKeypoints(crew_d, kp_cd, None,color=(255,0,0))
plt.imshow(img2)

### 5. Binary Robust Independent Elementary Features (BRIEF)

BRIEF calcule des descripteurs pour chaque point d'intérêt. Tout d'abord, un flou est effectué sur l'image d'entrée pour diminuer le niveau de bruit.
> from scipy.ndimage import gaussian_filter
> gaussian_filter(img, sigma=2)

On cherche alors à obtenir un descripteur, ici un vecteur de taille 128, pour chaque pixel détecté comme un point d'intérêt par FAST. On considère le voisinage de ce pixel (carré de taille 9x9). Dans ce voisinage, 128 paires de pixels (A,B) sont sélectionnées aléatoirement. Pour chacune de ces paires, si img(A) < img(B), alors le vecteur prend la valeur 1 (dans la case correspondant au numéro de la paire considérée). Dans le cas contraire, le vecteur prend la valeur 0.

### 6. Oriented FAST and Rotated BRIEF (ORB)
ORB combine le détecteur de points d'intérêt FAST et le descripteur binaire BRIEF en une seule méthode de détection et de description de caractéristiques.
- Calcul d'orientation pour chaque point d'intérêt (en détectant les coins et le centroïde)
- Mécanisme d'invariance à l'échelle en calculant les descripteurs à plusieurs résolutions 

In [ ]:
orb = cv2.ORB_create()

keypoints1, descriptors1 = orb.detectAndCompute(crew_d, None)
keypoints2, descriptors2 = orb.detectAndCompute(capsule, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

matches = bf.match(descriptors1, descriptors2)

matches = sorted(matches, key=lambda x: x.distance)

# Draw the first 10 matches
result_image = cv2.drawMatches(crew_d, keypoints1, capsule, keypoints2, matches[:20], outImg=None)

plt.imshow(result_image)
